Output URI
s3://groups-bucket-dblab-905418150721/group23/outputs/query2/

Να βρεθούν, για κάθε έτος, τα 3 Αστυνομικά Τμήματα με το υψηλότερο ποσοστό κλεισμένων (περατω-
μένων) υποθέσεων. Να τυπωθούν το έτος, τα ονόματα (τοποθεσίες) των τμημάτων, τα ποσοστά τους
καθώς και οι αριθμοί του ranking τους στην ετήσια κατάταξη. Τα αποτελέσματα να δοθούν σε σειρά
αύξουσα ως προς το έτος και το ranking 

Να υλοποιηθεί το Query 2 χρησιμοποιώντας τα DataFrame και SQL APIs. Να αναφέρετε και
να συγκρίνετε τους χρόνους εκτέλεσης μεταξύ των δύο υλοποιήσεων. 

θετουμε το configuration όπως το θέλει η εκφώνηση.

In [5]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "4g"
    }
}

Starting Spark application


The code failed because of a fatal error:
	Session 3986 did not start up in 60 seconds..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3839,application_1732639283265_3779,pyspark,idle,Link,Link,None,
3843,application_1732639283265_3783,pyspark,idle,Link,Link,None,
3850,application_1732639283265_3790,pyspark,idle,Link,Link,None,
3855,application_1732639283265_3795,pyspark,idle,Link,Link,None,
3872,application_1732639283265_3812,pyspark,idle,Link,Link,None,
3875,application_1732639283265_3815,pyspark,idle,Link,Link,None,
3882,application_1732639283265_3822,pyspark,idle,Link,Link,None,
3883,application_1732639283265_3823,pyspark,idle,Link,Link,None,
3887,application_1732639283265_3827,pyspark,idle,Link,Link,None,
3897,application_1732639283265_3837,pyspark,idle,Link,Link,None,


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Query2") \
    .getOrCreate()



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3828,application_1732639283265_3768,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Spark DataFrame code 

from pyspark.sql.functions import col, lower, udf
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank, desc
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import row_number
from pyspark.sql.functions import dense_rank
import csv
import time


# Define file paths
crime_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

print("Starting Query 2")

# --- DataFrame-based Implementation ---
start_time_df = time.time()

# Load datasets
crime_2019 = spark.read.csv(crime_2019_path, header=True)
crime_2020 = spark.read.csv(crime_2020_path, header=True)

# Combine datasets
crime_data = crime_2019.union(crime_2020)

print("Sample of Date Rptd values:")
crime_data.select("Date Rptd").distinct().show(10, truncate=False)

# Add a year column
crime_data = crime_data.withColumn("Year", col("Date Rptd").substr(7, 4))


# Filter and calculate closed case rates, we need both the closed and all of them
closed_cases = crime_data.filter(((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont")) & col("Year").isNotNull())
total_cases = crime_data.filter(col("Year").isNotNull())

crime_data.groupBy("Year").agg(countDistinct("Area Name")).show()


# This is the number of closed cases per year and area
closed_case_rates = closed_cases.groupBy(col("Year"), col("Area Name")).count() \
    .withColumnRenamed("count", "closed_count")

# This is the number of cases per year and area
area_totals = total_cases.groupBy(col("Year"), col("Area Name")).count() \
    .withColumnRenamed("count", "total_count")

# This is the rate of closed cases per year and area
rate_df = closed_case_rates.join(area_totals, ["Year", "Area Name"], how="inner") \
    .withColumn("closed_case_rate", (col("closed_count") / col("total_count")) * 100)

# We partition to order in descending order over the year
window_spec = Window.partitionBy("Year").orderBy(desc("closed_case_rate"))
# We calculate the rank
ranked_df = rate_df.withColumn("rank", row_number().over(window_spec))


# Add rankings per year (This was a test)
# rank_test = rate_df.withColumn("rnk", dense_rank().over(window_spec))
# rank_test.orderBy("Year", desc("closed_case_rate")).show(1000, truncate=False)

# Filter for top 3 per year
top3_df = ranked_df.filter(col("rank") <= 3).orderBy("Year", "rank")

# Rename columns to match example output
formatted_df = top3_df.select(
    col("Year").alias("year"),
    col("Area Name").alias("precinct"),
    col("closed_case_rate"),
    col("rank").alias("#")
)
# Show results
print("DataFrame Results:")
top3_df.select("Year", "Area Name", "closed_case_rate", "rank").show(1000, truncate=False)

end_time_df = time.time()
df_elapsed_time = end_time_df - start_time_df
print(f"DataFrame Time taken: {df_elapsed_time:.2f} seconds")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting Query 2
Sample of Date Rptd values:
+----------------------+
|Date Rptd             |
+----------------------+
|11/12/2013 12:00:00 AM|
|01/18/2013 12:00:00 AM|
|03/13/2022 12:00:00 AM|
|03/19/2013 12:00:00 AM|
|09/22/2021 12:00:00 AM|
|12/24/2012 12:00:00 AM|
|04/11/2013 12:00:00 AM|
|09/14/2012 12:00:00 AM|
|01/22/2012 12:00:00 AM|
|12/01/2014 12:00:00 AM|
+----------------------+
only showing top 10 rows

+----+-------------------------+
|Year|count(DISTINCT Area Name)|
+----+-------------------------+
|2020|                       21|
|2014|                       21|
|2018|                       21|
|2022|                       21|
|2023|                       21|
|2010|                       21|
|2021|                       21|
|2015|                       21|
|2011|                       21|
|2024|                       21|
|2016|                       21|
|2012|                       21|
|2019|                       21|
|2017|                       21|
|2013|            

Now lets test the implementation in SQL code. 

We can achieve the same result in SQL by:

    *Creating or replacing a temporary view (crime_view) from the (already filtered!) crime_data.
    *Running a CTE query with closed_cases, total_cases, and a rank window function.

In [4]:
import time
from pyspark.sql.functions import to_date, year, col

# --------------------------------------------------------------------------------
# 1. Assuming "crime_data" is already loaded and combined as above.
# --------------------------------------------------------------------------------
start_time_sql = time.time()

# Load datasets
crime19 = spark.read.csv(crime_2019_path, header=True)
crime20 = spark.read.csv(crime_2020_path, header=True)

# --------------------------------------------------------------------------------
# 2. Create a temp view for querying via Spark SQL
# --------------------------------------------------------------------------------
crime19.createOrReplaceTempView("crime19")
crime20.createOrReplaceTempView("crime20")


# --------------------------------------------------------------------------------
# 3. SQL Query using CTEs
# --------------------------------------------------------------------------------
sql_query = """
    WITH crime_view AS (
        SELECT * FROM crime19
        UNION
        SELECT * FROM crime20
    ),
    crime_with_year AS (
        SELECT *, substr(`Date Rptd`, 7, 4) AS Year
        FROM crime_view
        WHERE substr(`Date Rptd`, 7, 4) IS NOT NULL
    ),
    closed_cases AS (
        SELECT 
            Year,
            `Area Name` AS precinct,
            COUNT(*) AS closed_count
        FROM crime_with_year
        WHERE `Status Desc` != 'UNK'
          AND `Status Desc` != 'Invest Cont'
        GROUP BY Year, `Area Name`
    ),
    total_cases AS (
        SELECT 
            Year,
            `Area Name` AS precinct,
            COUNT(*) AS total_count
        FROM crime_with_year
        GROUP BY Year, `Area Name`
    )
SELECT
    sub.Year AS year,
    sub.precinct,
    (sub.closed_count * 100.0 / sub.total_count) AS closed_case_rate,
    sub.rnk AS `#`
FROM (
    SELECT
        c.Year,
        c.precinct,
        c.closed_count,
        t.total_count,
        ROW_NUMBER() OVER (
            PARTITION BY c.Year
            ORDER BY (c.closed_count * 1.0 / t.total_count) DESC
        ) AS rnk
    FROM closed_cases c
    JOIN total_cases t
      ON c.Year     = t.Year
     AND c.precinct = t.precinct
) sub
WHERE sub.rnk <= 3
ORDER BY sub.Year, sub.rnk
"""

# 4) Execute the SQL and display the results
sql_result = spark.sql(sql_query)
sql_result.show(50, truncate=False)

# 5) End timing for SQL approach
end_time_sql = time.time()
sql_elapsed_time = end_time_sql - start_time_sql
print(f"SQL Time taken: {sql_elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+-----------------+---+
|year|precinct   |closed_case_rate |#  |
+----+-----------+-----------------+---+
|2010|Rampart    |32.94735585531813|1  |
|2010|Olympic    |31.96270619172842|2  |
|2010|Harbor     |29.63203463203463|3  |
|2011|Olympic    |35.21216768916155|1  |
|2011|Rampart    |32.51177963030083|2  |
|2011|Harbor     |28.65220520201501|3  |
|2012|Olympic    |34.41481831052383|1  |
|2012|Rampart    |32.94641810294290|2  |
|2012|Harbor     |29.81513327601032|3  |
|2013|Olympic    |33.52812271731191|1  |
|2013|Rampart    |32.08287360549222|2  |
|2013|Harbor     |29.16422459266206|3  |
|2014|Van Nuys   |31.80567315834039|1  |
|2014|West Valley|31.31198995605775|2  |
|2014|Mission    |31.16279069767442|3  |
|2015|Van Nuys   |32.64134698172773|1  |
|2015|West Valley|30.27597402597403|2  |
|2015|Mission    |30.17946067838016|3  |
|2016|Van Nuys   |31.88075572011773|1  |
|2016|West Valley|31.54798761609907|2  |
|2016|Foothill   |29.87029184335246|3  |
|2017|Van Nuys  

In [5]:
print(f"DataFrame Time taken: {df_elapsed_time:.2f} seconds")
print(f"SQL Time taken: {sql_elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame Time taken: 38.84 seconds
SQL Time taken: 19.90 seconds

# Ερώτημα β

In [6]:

csv_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
csv_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

# Load datasets
crime_2019 = spark.read.csv(crime_2019_path, header=True)
crime_2020 = spark.read.csv(crime_2020_path, header=True)

# Combine datasets
crime_data = crime_2019.union(crime_2020)

# output URI

output_URI = "s3://groups-bucket-dblab-905418150721/group23/outputs/query2/"

# Save the combined DataFrame as a single Parquet file
crime_data.coalesce(1).write.mode("overwrite").parquet(output_URI)

print(f"Data successfully written as a single Parquet file to {output_URI}.")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data successfully written as a single Parquet file to s3://groups-bucket-dblab-905418150721/group23/outputs/query2/.

In [4]:
from pyspark.sql.functions import col, countDistinct, desc, row_number
from pyspark.sql.window import Window
import time

# Start timer
start_time_df = time.time()


parquet_file_path = "s3://groups-bucket-dblab-905418150721/group23/outputs/query2/part-00000-9934d31d-7771-49a9-97e2-e54e63b10046-c000.snappy.parquet"

# Load the Parquet file
crime_data = spark.read.parquet(parquet_file_path)

print("Sample of Date Rptd values:")
crime_data.select("Date Rptd").distinct().show(10, truncate=False)

# Add a year column
crime_data = crime_data.withColumn("Year", col("Date Rptd").substr(7, 4))


# Filter and calculate closed case rates, we need both the closed and all of them
closed_cases = crime_data.filter(((col("Status Desc") != "UNK") & (col("Status Desc") != "Invest Cont")) & col("Year").isNotNull())
total_cases = crime_data.filter(col("Year").isNotNull())

crime_data.groupBy("Year").agg(countDistinct("Area Name")).show()


# This is the number of closed cases per year and area
closed_case_rates = closed_cases.groupBy(col("Year"), col("Area Name")).count() \
    .withColumnRenamed("count", "closed_count")

# This is the number of cases per year and area
area_totals = total_cases.groupBy(col("Year"), col("Area Name")).count() \
    .withColumnRenamed("count", "total_count")

# This is the rate of closed cases per year and area
rate_df = closed_case_rates.join(area_totals, ["Year", "Area Name"], how="inner") \
    .withColumn("closed_case_rate", (col("closed_count") / col("total_count")) * 100)

# We partition to order in descending order over the year
window_spec = Window.partitionBy("Year").orderBy(desc("closed_case_rate"))
# We calculate the rank
ranked_df = rate_df.withColumn("rank", row_number().over(window_spec))


# Add rankings per year (This was a test)
# rank_test = rate_df.withColumn("rnk", dense_rank().over(window_spec))
# rank_test.orderBy("Year", desc("closed_case_rate")).show(1000, truncate=False)

# Filter for top 3 per year
top3_df = ranked_df.filter(col("rank") <= 3).orderBy("Year", "rank")

# Rename columns to match example output
formatted_df = top3_df.select(
    col("Year").alias("year"),
    col("Area Name").alias("precinct"),
    col("closed_case_rate"),
    col("rank").alias("#")
)
# Show results
print("DataFrame Results:")
top3_df.select("Year", "Area Name", "closed_case_rate", "rank").show(1000, truncate=False)

end_time_df = time.time()
df_elapsed_time = end_time_df - start_time_df
print(f"DataFrame Time taken: {df_elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sample of Date Rptd values:
+----------------------+
|Date Rptd             |
+----------------------+
|08/25/2011 12:00:00 AM|
|11/12/2013 12:00:00 AM|
|02/08/2020 12:00:00 AM|
|11/15/2021 12:00:00 AM|
|12/18/2021 12:00:00 AM|
|10/20/2017 12:00:00 AM|
|11/15/2019 12:00:00 AM|
|01/18/2013 12:00:00 AM|
|03/13/2022 12:00:00 AM|
|11/27/2023 12:00:00 AM|
+----------------------+
only showing top 10 rows

+----+-------------------------+
|Year|count(DISTINCT Area Name)|
+----+-------------------------+
|2020|                       21|
|2014|                       21|
|2018|                       21|
|2022|                       21|
|2023|                       21|
|2010|                       21|
|2021|                       21|
|2015|                       21|
|2011|                       21|
|2024|                       21|
|2016|                       21|
|2012|                       21|
|2019|                       21|
|2017|                       21|
|2013|                       21|
+-

In [8]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, rank
from pyspark.sql.window import Window

# 1. Αρχικοποίηση SparkSession
spark = SparkSession.builder \
    .appName("CSVvsParquet") \
    .getOrCreate()

# 2. S3 paths 
csv_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
csv_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
parquet_output_path = "s3://buckets/groups-bucket-dblab-905418150721/group23/outputs/query2/output_parquet_singlefile"  # Εδώ θα γράψουμε 1 .parquet
parquet_input_path  = "s3://buckets/groups-bucket-dblab-905418150721/group23/outputs/query2/output_parquet_singlefile"  # Για να το ξαναφορτώσουμε

# ----------------------------------------------------------------------------
# Function for "top 3 closed cases per year"
# ----------------------------------------------------------------------------
def compute_top3_closed(df):
    """
    df: DataFrame με columns: [Date Rptd, Status Desc, Area Name, ...]
    Επιστρέφει DataFrame με Year, Area Name, closed_case_rate, rank
    όπου "closed" = οτιδήποτε εκτός από "UNK" και "Invest Cont"
    και total = όλα τα records.
    """
    
    # Εξαγωγή έτους απλό, π.χ. substr(7,4) ή καλύτερα χρήση regexp/ημερομηνίας
    df = df.withColumn("Year", col("Date Rptd").substr(7,4))
    
    # Closed = Status Desc != 'UNK' & != 'Invest Cont'
    closed_df = df.filter((col("Status Desc") != "UNK") &
                          (col("Status Desc") != "Invest Cont"))
    total_df  = df  # όλα
    
    # Μετράμε closed / total per (Year, Area Name)
    closed_agg = closed_df.groupBy("Year", "Area Name").count().alias("closed_count")
    closed_agg = closed_agg.withColumnRenamed("count", "closed_count")

    total_agg = total_df.groupBy("Year", "Area Name").count().alias("total_count")
    total_agg = total_agg.withColumnRenamed("count", "total_count")
    
    # Join και υπολογισμός ποσοστού
    joined = closed_agg.join(total_agg, ["Year", "Area Name"], how="inner") \
                       .withColumn("closed_case_rate",
                                   (col("closed_count")*100.0 / col("total_count")))
    
    # Παράθυρο για κατάταξη top 3 ανά Year
    w = Window.partitionBy("Year").orderBy(desc("closed_case_rate"))
    ranked = joined.withColumn("rank", rank().over(w))
    
    top3 = ranked.filter(col("rank") <= 3).orderBy("Year", "rank")
    return top3

# ----------------------------------------------------------------------------
# A) Φόρτωση από CSV & υπολογισμός χρόνου
# ----------------------------------------------------------------------------
start_time_csv = time.time()

csv_2019_df = spark.read.csv(csv_2019_path, header=True, inferSchema=True)
csv_2020_df = spark.read.csv(csv_2020_path, header=True, inferSchema=True)

df_csv = csv_2019_df.union(csv_2020_df)

# Ερώτημα top3
top3_from_csv = compute_top3_closed(df_csv)

# Δείξε αποτελέσματα (προαιρετικά)
top3_from_csv.show(10, truncate=False)

end_time_csv = time.time()
csv_elapsed = end_time_csv - start_time_csv
print(f"Time (load CSV + compute query) = {csv_elapsed:.2f} sec")

# ----------------------------------------------------------------------------
# B) Αποθήκευση σε Parquet (ένα μόνο αρχείο) & Επαναφόρτωση
# ----------------------------------------------------------------------------
# Παράδειγμα αποθήκευσης ως Parquet σε ΕΝΑ αρχείο:
df_csv.coalesce(1).write.mode("overwrite").parquet(parquet_output_path)
# Το coalesce(1) αναγκάζει τη δημιουργία ενός μόνο part αρχείου. 
# Μεγάλα datasets -> δεν συνιστάται για production, αλλά οκ για την άσκηση.

# ----------------------------------------------------------------------------
# C) Φόρτωση από Parquet & υπολογισμός χρόνου
# ----------------------------------------------------------------------------
start_time_parquet = time.time()

df_parquet = spark.read.parquet(parquet_input_path)
top3_from_parquet = compute_top3_closed(df_parquet)
top3_from_parquet.show(10, truncate=False)

end_time_parquet = time.time()
parquet_elapsed = end_time_parquet - start_time_parquet
print(f"Time (load Parquet + compute query) = {parquet_elapsed:.2f} sec")

# ----------------------------------------------------------------------------
# D) Σύγκριση Χρόνων
# ----------------------------------------------------------------------------
print("===========================================")
print(f"CSV total time:      {csv_elapsed:.2f} sec")
print(f"Parquet total time:  {parquet_elapsed:.2f} sec")
print("===========================================")

spark.stop()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o619.parquet.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: 2E55JT6FDG525ZY5; S3 Extended Request ID: I/MX4tzHK6Ww5ODRPNYznvrs487U2TaRATmEH8Htf1Gqb+aVzt0mBv8M7tVufUj6s/XepfkYhJc=; Proxy: null), S3 Extended Request ID: I/MX4tzHK6Ww5ODRPNYznvrs487U2TaRATmEH8Htf1Gqb+aVzt0mBv8M7tVufUj6s/XepfkYhJc=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:429)
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.isFolderUsingFolderObject(Jets3tNativeFileSystemStore.java:255)
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.isFolder(Jets3tNativeFileSystemStore.java:218)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:554)
	at org.apache.hadoop.fs.FileSystem.exists(FileSy